# Preços de imóveis em João Pessoa

## Introdução

Esta análise se baseia na raspagem de dados na OLX para a pesquisa <a href = "https://www.olx.com.br/brasil?q=im%C3%B3veis%20em%20jo%C3%A3o%20pessoa&o=1">imóveis em joão pessoa</a>, seguida de análise exploratória desses dados.

São empregadas para tanto as seguintes bibliotecas:

- Requests para a realização de requisições HTTP;

- time para intervalos entre requisições;

- random para randomizar intervalos(otimização);

- tqdm para controle de tempo;

- BeautifulSoup para a extração de dados do html das páginas;

- re para a filtragem mais fina desses dados;

- Pandas para manipulação de dados em dataframes.

## 1. Importar bibliotecas

In [88]:
import requests
import time
from random import uniform
from tqdm import tqdm
from bs4 import BeautifulSoup
import re
import pandas as pd
import csv

## 2. Web scraping

A primeira parte de nossa análise será a extração de dados por scraping do site da OLX. Para tanto procederemos assim:

1. Inicialmente varremos as páginas de pesquisa à procura dos links de cada um dos produtos;

2. Depois, para cada produto, faremos a raspagem de suas informações em sua própria página.

In [2]:
base_url = 'https://www.olx.com.br/brasil?q=im%C3%B3veis%20em%20jo%C3%A3o%20pessoa'

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

products_links = []
max_consecutive_empty_pages = 3
consecutive_empty_pages = 0
page = 0

# Criando o tqdm apenas para mostrar o tempo estimado
pbar = tqdm(total=100, desc="Processando", unit="página", bar_format='{desc}: {n_fmt}/{total_fmt} [Tempo restante: {remaining}]')

while True:
    page += 1
    
    url = f"{base_url}&o={page}"

    try:
        search_response = requests.get(url, headers=headers)

        if search_response.status_code == 200:
            search_soup = BeautifulSoup(search_response.text, 'html.parser')

            no_results_element = search_soup.find('div', class_='empty-list')
            if no_results_element or 'Nenhum resultado encontrado' in search_response.text:
                pbar.set_postfix_str("Não há mais resultados disponíveis")
                break

            all_links = search_soup.find_all('a', class_='olx-adcard__link')
            
            if all_links:
                consecutive_empty_pages = 0
                for link in all_links:
                    href = link.get('href')
                    if href and href.startswith('http'):
                        products_links.append(href)
            else:
                consecutive_empty_pages += 1
                pbar.set_postfix_str(f"Pág {page} vazia ({consecutive_empty_pages} consecutivas)")
                
                if consecutive_empty_pages >= max_consecutive_empty_pages:
                    pbar.set_postfix_str(f"Parando após {max_consecutive_empty_pages} páginas vazias")
                    break

        elif search_response.status_code == 404:
            pbar.set_postfix_str("Página não encontrada (404)")
            break
        else:
            pbar.set_postfix_str(f"Erro: {search_response.status_code}")
            
    except Exception as e:
        pbar.set_postfix_str(f"Erro: {e}")
        
    if page >= 100:
        pbar.set_postfix_str("Limite de 100 páginas atingido")
        break

    # Atualizar o progresso para calcular o tempo restante
    pbar.update(1)
    pbar.set_postfix_str(f"Produtos: {len(products_links)}")
    
    time.sleep(uniform(1, 3))

pbar.close()
print(f"Processamento concluído! Total de produtos encontrados: {len(products_links)}")

Processando: 99/100 [Tempo restante: 00:03]

Processamento concluído! Total de produtos encontrados: 5000


Vejamos os dez primeiros links que coletamos:

In [3]:
products_links[0:10]

['https://pb.olx.com.br/paraiba/imoveis/imoveis-na-planta-em-joao-pessoa-1427390473',
 'https://pb.olx.com.br/paraiba/imoveis/vendo-imoveis-de-alto-padrao-a-beira-mar-de-intermares-1399963055',
 'https://pb.olx.com.br/paraiba/imoveis/imoveis-na-planta-1424880641',
 'https://pb.olx.com.br/paraiba/imoveis/imoveis-exclusivos-1422666330',
 'https://pb.olx.com.br/paraiba/servicos/imoveis-vende-ou-aluga-casa-apartamento-terreno-1348330489',
 'https://pb.olx.com.br/paraiba/imoveis/alugo-imoveis-1418639984',
 'https://pb.olx.com.br/paraiba/imoveis/apartamentos-para-venda-varios-imoveis-1425816782',
 'https://pb.olx.com.br/paraiba/servicos/reforma-de-imoveis-em-joao-pessoa-pisos-pintura-e-outros-acabamentos-1424786399',
 'https://pb.olx.com.br/paraiba/servicos/corretor-de-imoveis-em-joao-pessoa-1357257740',
 'https://pb.olx.com.br/paraiba/servicos/seu-corretor-de-imoveis-perito-avaliador-em-joao-pessoa-1385905983']

São exatamente os dez links que aparecem na primeira página de busca! Vamos agora investigá-los individualmente:

In [4]:
all_products = []

# Criando a barra de progresso tqdm
pbar = tqdm(total=len(products_links), desc="Processando produtos", unit="produto", bar_format='{desc}: {n_fmt}/{total_fmt} [Tempo restante: {remaining}]')

for i, link in enumerate(products_links, 1):
    try:
        product_response = requests.get(link, headers=headers, timeout=10)

        if product_response.status_code == 200:
            product_soup = BeautifulSoup(product_response.text, 'html.parser')

            title_elem = product_soup.find('span', class_='olx-text olx-text--title-medium olx-text--block ad__sc-1l883pa-2 bdcWAn')
            title = title_elem.text.strip() if title_elem else 'Não especificado'

            location_div = product_soup.find('div', id='location')
            if location_div:
                location_elem = location_div.find('span', class_='olx-text olx-text--body-small olx-text--block olx-text--semibold olx-color-neutral-110')
                location = location_elem.text.strip() if location_elem else 'Não especificado'
            else:
                location = 'Não especificado'
            
            price_div = product_soup.find('div', id='price-box-container')
            if price_div:
                price_elem = price_div.find('span', {'data-ds-component': 'DS-Text'})
                price = price_elem.text.strip() if price_elem else 'Não especificado'
                
                product_type_elem = price_div.find('span', class_='olx-badge olx-badge--neutral olx-badge--small olx-badge--rectangle')
                product_type = product_type_elem.text.strip() if product_type_elem else "Não especificado"
            else:
                price = 'Não especificado'
                product_type = "Não especificado"

            product_data = {
                'title': title,
                'location': location,
                'price': price,
                'product_type': product_type,
                'link': link  
            }

            all_products.append(product_data)
            
            # Atualizar informações na barra de progresso
            pbar.set_postfix_str(f"Último: {title[:20]}...")

        else:
            pbar.set_postfix_str(f"Erro: Status {product_response.status_code}")

    except Exception as e:
        pbar.set_postfix_str(f"Erro: {str(e)[:20]}...")
    
    # Atualizar o progresso
    pbar.update(1)
    time.sleep(uniform(2, 4))

pbar.close()
print(f"Extração concluída! Total de produtos: {len(all_products)}")

Processando produtos: 1015/5000 [Tempo restante: 7824:42:54]
KeyboardInterrupt



In [ ]:
all_products[0:10]

Estamos perto de um conjunto de dados minimamente bem tratado antes de começar a manipulação mais refinada deles. O que observamos é que temos elementos com dados faltantes na coluna de preços, e que esses, porém, são justamente o que **não** queremos incluir na nossa análise (corretores de imóveis se autopromovendo etc.). Isso, porém, pode e será corrigido na etapa de limpeza e tratamento de dados.

Assim, passemos ao território do Pandas.

## 3. Tratamento e limpeza de dados

In [76]:
df = pd.DataFrame(all_products)

In [77]:
df.head()

,title,location,price,product_type,link
0,Imóveis na planta em João Pessoa,"Cabo Branco, João Pessoa, PB, 58045010",R$ 274.900,Venda,https://pb.olx.com.br/paraiba/imoveis/imoveis-...
1,Vendo Imóveis de Alto padrão a Beira Mar de In...,"Ipês, João Pessoa, PB, 58028530",R$ 480.000,Venda,https://pb.olx.com.br/paraiba/imoveis/vendo-im...
2,Imóveis na Planta,"Jardim Oceania, João Pessoa, PB, 58037030",R$ 416.234,Venda,https://pb.olx.com.br/paraiba/imoveis/imoveis-...
3,Imoveis exclusivos,"Manaíra, João Pessoa, PB, 58038142",R$ 650.000,Venda,https://pb.olx.com.br/paraiba/imoveis/imoveis-...
4,IMOVEIS VENDE OU ALUGA CASA APARTAMENTO TERRENO,"João Pessoa, PB, 58052030",Não especificado,Não especificado,https://pb.olx.com.br/paraiba/servicos/imoveis...


In [78]:
df.tail()

,title,location,price,product_type,link
950,Oportunidade Única em JOAO PESSOA - PB | Tipo:...,"João Pessoa, PB, 58069260",R$ 79.360,Venda,https://pb.olx.com.br/paraiba/imoveis/oportuni...
951,Oportunidade Única em JOAO PESSOA - PB | Tipo:...,"João Pessoa, PB, 58066100",R$ 79.939,Venda,https://pb.olx.com.br/paraiba/imoveis/oportuni...
952,Oportunidade Única em JOAO PESSOA - PB | Tipo:...,"João Pessoa, PB, 58062349",R$ 69.889,Venda,https://pb.olx.com.br/paraiba/imoveis/oportuni...
953,Oportunidade Única em JOAO PESSOA - PB | Tipo:...,"João Pessoa, PB, 58052130",R$ 121.021,Venda,https://pb.olx.com.br/paraiba/imoveis/oportuni...
954,Oportunidade Única em JOAO PESSOA - PB | Tipo:...,"João Pessoa, PB, 58080020",R$ 71.080,Venda,https://pb.olx.com.br/paraiba/imoveis/oportuni...


Vamos analisar os tipos de dados:

In [79]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 955 entries, 0 to 954
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title         955 non-null    object
 1   location      955 non-null    object
 2   price         955 non-null    object
 3   product_type  955 non-null    object
 4   link          955 non-null    object
dtypes: object(5)
memory usage: 37.4+ KB


Vamos logo resolver a questão da limpeza de dados quanto aos preços:

In [80]:
price_regex = r'R\$\s*\d{1,3}(?:\.d{3})*(?:\,d{2})?'

invalid_mask = ~df['price'].apply(lambda x: bool(re.search(price_regex, str(x), re.IGNORECASE)))
df.drop(df[invalid_mask].index, inplace=True)

df['price'] = df['price'].str.replace('R$ ', '')
df['price'] = df['price'].str.replace('.', '')
df['price'] = df['price'].str.replace(',', '.')
df['price'] = df['price'].astype('float64')

In [81]:
df

,title,location,price,product_type,link
0,Imóveis na planta em João Pessoa,"Cabo Branco, João Pessoa, PB, 58045010",274900.0,Venda,https://pb.olx.com.br/paraiba/imoveis/imoveis-...
1,Vendo Imóveis de Alto padrão a Beira Mar de In...,"Ipês, João Pessoa, PB, 58028530",480000.0,Venda,https://pb.olx.com.br/paraiba/imoveis/vendo-im...
2,Imóveis na Planta,"Jardim Oceania, João Pessoa, PB, 58037030",416234.0,Venda,https://pb.olx.com.br/paraiba/imoveis/imoveis-...
3,Imoveis exclusivos,"Manaíra, João Pessoa, PB, 58038142",650000.0,Venda,https://pb.olx.com.br/paraiba/imoveis/imoveis-...
5,ALUGO IMÓVEIS,"Treze de Maio, João Pessoa, PB, 58025430",800.0,Aluguel,https://pb.olx.com.br/paraiba/imoveis/alugo-im...
...,...,...,...,...,...
950,Oportunidade Única em JOAO PESSOA - PB | Tipo:...,"João Pessoa, PB, 58069260",79360.0,Venda,https://pb.olx.com.br/paraiba/imoveis/oportuni...
951,Oportunidade Única em JOAO PESSOA - PB | Tipo:...,"João Pessoa, PB, 58066100",79939.0,Venda,https://pb.olx.com.br/paraiba/imoveis/oportuni...
952,Oportunidade Única em JOAO PESSOA - PB | Tipo:...,"João Pessoa, PB, 58062349",69889.0,Venda,https://pb.olx.com.br/paraiba/imoveis/oportuni...
953,Oportunidade Única em JOAO PESSOA - PB | Tipo:...,"João Pessoa, PB, 58052130",121021.0,Venda,https://pb.olx.com.br/paraiba/imoveis/oportuni...


In [94]:
df['title'] = df['title'].apply(lambda x: x.lower())

In [83]:
df['location'] = df['location'].str.extract(r'(\d{8})')

In [84]:
df = df.rename(columns = {'location': 'cep'})

In [85]:
df

,title,cep,price,product_type,link
0,imóveis na planta em joão pessoa,58045010,274900.0,Venda,https://pb.olx.com.br/paraiba/imoveis/imoveis-...
1,vendo imóveis de alto padrão a beira mar de in...,58028530,480000.0,Venda,https://pb.olx.com.br/paraiba/imoveis/vendo-im...
2,imóveis na planta,58037030,416234.0,Venda,https://pb.olx.com.br/paraiba/imoveis/imoveis-...
3,imoveis exclusivos,58038142,650000.0,Venda,https://pb.olx.com.br/paraiba/imoveis/imoveis-...
5,alugo imóveis,58025430,800.0,Aluguel,https://pb.olx.com.br/paraiba/imoveis/alugo-im...
...,...,...,...,...,...
950,oportunidade única em joao pessoa - pb | tipo:...,58069260,79360.0,Venda,https://pb.olx.com.br/paraiba/imoveis/oportuni...
951,oportunidade única em joao pessoa - pb | tipo:...,58066100,79939.0,Venda,https://pb.olx.com.br/paraiba/imoveis/oportuni...
952,oportunidade única em joao pessoa - pb | tipo:...,58062349,69889.0,Venda,https://pb.olx.com.br/paraiba/imoveis/oportuni...
953,oportunidade única em joao pessoa - pb | tipo:...,58052130,121021.0,Venda,https://pb.olx.com.br/paraiba/imoveis/oportuni...


In [93]:
df = df.replace(',', '')

In [100]:
df.to_csv('data.csv', sep = ',', escapechar = '\\')